In [5]:
import mlflow.azureml

from azureml.core import Workspace
from azureml.core.webservice import AciWebservice, Webservice


workspace_name = "aeamlws"
subscription_id = "d4e5fecf-32d0-4314-a56e-ca2389ac7ac3"
resource_group = "DataPlatformAESKORGDev"
location = "westeurope"

## Create AzureML Workspace

In [4]:
azure_workspace = Workspace.create(name=workspace_name,
                                   subscription_id=subscription_id,
                                   resource_group=resource_group,
                                   location=location,
                                   create_resource_group=True)

Deploying KeyVault with name mlflowdekeyvault0f6efcf8.
Deployment of StorageAccount with name mlflowdestorage73adc2d2e failed.
Deploying AppInsights with name mlflowdeinsights1a99c1a0.
Deployed AppInsights with name mlflowdeinsights1a99c1a0. Took 5.18 seconds.
Deployed KeyVault with name mlflowdekeyvault0f6efcf8. Took 17.81 seconds.
Unable to create the workspace. 
 Azure Error: DeploymentFailed
Message: At least one resource deployment operation failed. Please list deployment operations for details. Please see https://aka.ms/arm-debug for usage details.
Exception Details:
	Error Code: Conflict
	Message: {'error': {'code': 'MaxStorageAccountsCountPerSubscriptionExceeded', 'message': 'Subscription d4e5fecf-32d0-4314-a56e-ca2389ac7ac3 already contains 252 storage accounts in location northeurope and the maximum allowed is 250.'}}


WorkspaceException: WorkspaceException:
	Message: Unable to create the workspace. At least one resource deployment operation failed. Please list deployment operations for details. Please see https://aka.ms/arm-debug for usage details.
	InnerException Azure Error: DeploymentFailed
Message: At least one resource deployment operation failed. Please list deployment operations for details. Please see https://aka.ms/arm-debug for usage details.
Exception Details:
	Error Code: Conflict
	Message: {'error': {'code': 'MaxStorageAccountsCountPerSubscriptionExceeded', 'message': 'Subscription d4e5fecf-32d0-4314-a56e-ca2389ac7ac3 already contains 252 storage accounts in location northeurope and the maximum allowed is 250.'}}
	ErrorResponse 
{
    "error": {
        "message": "Unable to create the workspace. At least one resource deployment operation failed. Please list deployment operations for details. Please see https://aka.ms/arm-debug for usage details."
    }
}

## Get already created workspace

In [6]:
azure_workspace = Workspace.get(name=workspace_name,
                                   subscription_id=subscription_id,
                                   resource_group=resource_group)

In [7]:
azure_workspace.list(subscription_id)

{'aeamlws': [Workspace.create(name='aeamlws', subscription_id='d4e5fecf-32d0-4314-a56e-ca2389ac7ac3', resource_group='DataPlatformAESKORGDev')],
 'mlflow-demo-deploy': [Workspace.create(name='mlflow-demo-deploy', subscription_id='d4e5fecf-32d0-4314-a56e-ca2389ac7ac3', resource_group='DataPlatformGreenTeamRGDev')]}

# Build an AzureML container image for deployment

In [8]:
azure_image, azure_model = mlflow.azureml.build_image(model_uri="./mlruns/0/1de47d82f2de4b0d9211799899b8d764/artifacts/models",
                                                      workspace=azure_workspace,
                                                      description="Text Classifier",
                                                      synchronous=True)
# If your image build failed, you can access build logs at the following URI:
print("Access the following URI for build logs: {}".format(azure_image.image_build_log_uri))

OSError: No such file or directory: '.\mlruns\0\1de47d82f2de4b0d9211799899b8d764\artifacts\models'

## Deploy the container image to Azure Container Instances

In [21]:
webservice_deployment_config = AciWebservice.deploy_configuration()
webservice = Webservice.deploy_from_image(
                    image=azure_image, workspace=azure_workspace, name="mlflowdemo2")
webservice.wait_for_deployment()

print("Scoring URI is: %s", webservice.scoring_uri)

ACI service creation operation finished, operation "Succeeded"
Scoring URI is: %s http://0f8577e9-1454-4ec1-be5f-c1324d2a9a70.northeurope.azurecontainer.io/score


In [22]:
print(webservice.get_logs())

2019-09-18T10:00:30,028514981+00:00 - iot-server/run 
2019-09-18T10:00:30,027979079+00:00 - rsyslog/run 
2019-09-18T10:00:30,029014984+00:00 - gunicorn/run 
2019-09-18T10:00:30,127746876+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-09-18T10:00:36,852431487+00:00 - iot-server/finish 1 0
2019-09-18T10:00:36,948798664+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.6.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 43
generated new fontManager
Initializing logger
Starting up app insights client
Starting up request id generator
Starting up app insight hooks
Invoking user's init function
2019-09-18 10:01:40,950 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerExce

## Making new predictions
After the image deployment completes, requests can be posted via HTTP to the new ACI webservice's scoring URI.

In [23]:
scoring_uri = webservice.scoring_uri #"http://9e04c0c1-75ae-4001-8f16-770bb2120e0b.westus.azurecontainer.io/score"
scoring_uri

'http://0f8577e9-1454-4ec1-be5f-c1324d2a9a70.northeurope.azurecontainer.io/score'

In [26]:
import requests
import json
import pandas as pd

# `sample_input` is a JSON-serialized pandas DataFrame with the `split` orientation

def get_prediction(text_to_predict):
    sample_input = {"columns":["description"],
                    "data":text_to_predict
                   }
    print(json.dumps(sample_input))
    response = requests.post(
                  url=scoring_uri, data=json.dumps(sample_input),
                  headers={"Content-type": "application/json",
                          "format": "pandas-split"})
    print(response)
    response_json = json.loads(response.text)
    return response_json

In [27]:
get_prediction([["machine learning artificial intelligence algorithm neural network"],["test medicine medical surgery"]])

{"columns": ["description"], "data": [["machine learning artificial intelligence algorithm neural network"], ["test medicine medical surgery"]]}
<Response [200]>


[{'AD': 0.01813508487277221,
  'HF': 0.05229500496841805,
  'IE': 0.5908285560032751,
  'IV': 0.171604584509179,
  'MH': 0.03724132739813264,
  'NV': 0.03795602689232898,
  'OK': 0.04047057157734069,
  'SU': 0.05146884377855149},
 {'AD': 0.03331348685608118,
  'HF': 0.08991528644571575,
  'IE': 0.17199133283675266,
  'IV': 0.15039080192133317,
  'MH': 0.26034723724296127,
  'NV': 0.15060933788287126,
  'OK': 0.06344532369869275,
  'SU': 0.07998719311559377}]